In [10]:
import torch, os
import sys, cv2
import matplotlib.pyplot as plt

sys.path.append('C:/Users/owner/Documents/GitHub/Scalable-Hyperspectral-3D-Imaging')
from hyper_sl.utils import _constants as C

device = 'cuda:0'
binning = 2


### grid pattern 생성

#### Input pixel location where white dots exist

In [11]:
#  D/fine the size of the image
WIDTH = 1280 // binning
HEIGHT = 720 // binning

# Define the size of the dot and its center coordinate
DOT_SIZE = 5
# CENTER_COORDS = [(100, 100), (100, 200), (100, 300), (300, 100), (300, 200), (300, 300), (500, 100), (500, 200), (500, 300)]
CENTER_COORDS = [ (480, 180)]

# # Create a tensor to store the dots
dots = torch.zeros((HEIGHT, WIDTH))

# Loop through each center coordinate and set the pixels of the corresponding dot to 1

for center_x, center_y in CENTER_COORDS:
    x_start = center_x - DOT_SIZE // 2
    y_start = center_y - DOT_SIZE // 2
    x_end = x_start + DOT_SIZE
    y_end = y_start + DOT_SIZE
    dots[ y_start:y_end, x_start:x_end] = 1
    
# Convert the tensor to a PIL image
grid = dots

# Display the imageplt.imshow(grid)
# plt.imsave('./grid.png', grid.numpy())
cv2.imwrite('./dot_480_180.png', grid.numpy()*255.)

True

In [12]:
# Intrinsic of camera
cam_H, cam_W = 1536//binning, 2048//binning
# sensor_width_cam = 5.32 * 1e-3
cam_pitch = 3.45*1e-6*2
focal_length_cam = (1.75261231e+03)*cam_pitch # 12 mm

In [4]:
# Intrinsic of projector
proj_sensor_diag = C.PROJ_DIAG *1e-3
proj_H = 720 // binning
proj_W = 1280 // binning
sensor_width_proj = torch.sin(torch.atan2(torch.tensor(proj_H),torch.tensor(proj_H)))*proj_sensor_diag
proj_pitch = 5.73 *1e-6 * 2
# proj_pitch = (sensor_width_proj/ proj_H)
proj_focal_length = (1.04829625e+03)*proj_pitch
# proj_focal_length = 14*1e-3

In [5]:
cam_int = torch.tensor([[1.75261231e+03, 0.00000000e+00 ,4.99891309e+02],
 [0.00000000e+00 ,1.76071560e+03 , 4.15817088e+02],
 [0.00000000e+00 ,0.00000000e+00, 1.00000000e+00]], device= device)

In [6]:
proj_int = torch.tensor([[1.04829625e+03 , 0.00000000e+00 ,2.76519034e+02],
 [0.00000000e+00 ,1.04580837e+03  ,3.10408841e+02],
 [0.00000000e+00 ,0.00000000e+00, 1.00000000e+00]] , device= device)

proj_dist = torch.tensor([[-1.21868710e-02 , 5.29790580e-01, -2.81219998e-03, -1.95464376e-02,-5.36525586e+00]])

In [7]:
dst_grid = cv2.undistort(grid.numpy(), proj_int.detach().cpu().numpy(), proj_dist.numpy())
dst_grid = torch.tensor(dst_grid).to(device)

## Depth 알아내기

In [8]:
cam_H, cam_W, cam_pitch, focal_length_cam

(768, 1024, 6.9e-06, 0.012093024939)

In [9]:
proj_H, proj_W, proj_pitch, proj_focal_length, proj_sensor_diag, sensor_width_proj

(360, 640, 1.146e-05, 0.012013475025000001, 0.005842, tensor(0.0041))

In [10]:
torch.linalg.inv(proj_int)

tensor([[ 9.5393e-04,  0.0000e+00, -2.6378e-01],
        [ 0.0000e+00,  9.5620e-04, -2.9681e-01],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')

##### Unprojection

In [11]:
def unprojection(depth, r, c):
        """ Unproject camera sensor coord plane to world coord

            input : depth
            return : world coordinate X,Y,Z
            
        """
        # c, r = torch.meshgrid(torch.linspace(0,proj_H-1,proj_H), torch.linspace(0,proj_W-1,proj_W), indexing='ij') # 행렬 indexing        
        # c, r = c.reshape(proj_W*proj_H), r.reshape(proj_W*proj_H)
        ones = torch.ones_like(r, device=device)
        
        xyz_p = torch.stack((r*depth,c*depth,ones*depth), dim = 0)
        
        XYZ = torch.linalg.inv(proj_int)@xyz_p
        
        return XYZ, r, c

##### extrinsic matrix of projector

In [12]:
def extrinsic_proj_real():
        """ World coordinate to real proj's coordinate
        
        """
        extrinsic_proj_real = torch.zeros((4,4), device= device)
        
        
        extrinsic_proj_real[:3,:3] = torch.tensor([[ 0.99401968,  0.01234548 , 0.10850096],
                                                        [-0.00400605,  0.99704272, -0.0767448 ],
                                                        [-0.10912754, 0.07585118  ,0.99112955]])

        t_mtrx = torch.tensor([[-65.40436873],
                                [-14.71175827],
                                [24.13928588]])
        
        extrinsic_proj_real[:3,3:4] = t_mtrx*1e-3
        extrinsic_proj_real[3,3] = 1
        
        extrinsic_proj_real = torch.linalg.inv(extrinsic_proj_real)[:3]
        return extrinsic_proj_real

##### Projection

In [13]:
def projection(XYZ):
        """
            proj coord to world coord
        """
        ones = torch.ones(size=(1,XYZ.shape[1]), device= device)
        
        XYZ1 = torch.concat((XYZ, ones), dim = 0)

        # XYZ coords in world coordinate               
        uv_cam = (cam_int)@(extrinsic_proj_real())@XYZ1.to(device= device)
        uv_cam = uv_cam / uv_cam[2]

        return uv_cam

##### find the idx of white camera pixel

In [14]:
def find_idx(xy_cam, illum, r, c):
    
    r_cam, c_cam = xy_cam[1], xy_cam[0]

    cond = (0<= r_cam)*(r_cam < cam_H)*(0<=c_cam)*(c_cam< cam_W)

    r_cam_valid, c_cam_valid = r_cam[cond], c_cam[cond]
    r_cam_valid, c_cam_valid = torch.tensor(r_cam_valid), torch.tensor(c_cam_valid)

    new_idx = cam_W * r_cam_valid.long() + c_cam_valid.long()      

    # captured_img = torch.zeros(size=(cam_H*cam_W,))

    # illum = illum.reshape(proj_H*proj_W)
    
    # proj_r, proj_c = r[cond], c[cond]
    # new_idx_illum = proj_W * proj_c.long() + proj_r.long()
    
    # captured_img[new_idx] = illum.reshape(proj_H* proj_W)[new_idx_illum]
    
    # captured_img = captured_img.reshape(cam_H, cam_W)
    
    # return captured_img, new_idx
    
    return new_idx

In [15]:
# depth = torch.zeros(size=(proj_H* proj_W,))

# depth[:] = 1.7

# r = torch.tensor([100, 300, 500, 100, 300, 500, 100, 300, 500], device=device)
# c = torch.tensor([100, 200, 300, 100, 200, 300, 100, 200, 300], device=device)

# # Unproject to 3D points
# XYZ,r,c = unprojection(depth=depth,r,c)

# # Projector to projector
# uv_cam = projection(XYZ)

# # new_idx = find_idx(xy_cam, torch.tensor(dst_grid), r, c)

In [16]:
# cam_int = torch.tensor([[1.75261231e+03, 0.00000000e+00 ,4.99891309e+02],
#  [0.00000000e+00 ,1.76071560e+03, 4.15817088e+02],
#  [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

cam_dist = torch.tensor([[-2.44349013e-01, 6.10520025e-01, -7.59154684e-04 ,-1.58477294e-03, -5.52344502e+00]])

In [17]:
# dst = cv2.undistort(captured_img.numpy(), cam_int.numpy(), cam_dist.numpy())

In [18]:
def find_idx_real():  
    real_x = torch.tensor([89, 428, 763, 82, 424, 762, 76, 423, 764], device = 'cuda:0') # 윗줄부터 아랫줄 찬찬히
    real_y = torch.tensor([207, 210, 216, 374, 379, 282, 546, 522, 552], device = 'cuda:0')
    ones = torch.ones_like(real_x, device=device)
    
    real_xy1 = torch.stack((real_x, real_y, ones), dim = 0)
    
    return real_xy1

### Depth value to be optimized

In [19]:
# optimize depth value

r = torch.tensor([100, 300, 500, 100, 300, 500, 100, 300, 500], device=device)
c = torch.tensor([100, 200, 300, 100, 200, 300, 100, 200, 300], device=device)

epoch = 10000
loss_f = torch.nn.L1Loss()

lr = 1e-2
decay_step = 1000

losses = []

depth_value = torch.rand(1, requires_grad = True, device = device)

print('initialized depth value : ', depth_value)

optimizer = torch.optim.Adam([depth_value], lr = lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=decay_step, gamma = 0.5)

for i in range(epoch):
    # depth value
    depth = torch.zeros(size=(r.shape[0],), device=device)
    depth[:] = depth_value
    # depth[:] = torch.tensor([1.7], requires_grad=True, device= device)

    # Unproject to 3D points
    XYZ, r,c = unprojection(depth=depth, r=r, c=c)

    # Projector to projector
    uv_cam = projection(XYZ)

    # find camera index
    # new_idx = find_idx(xy_cam, torch.tensor(dst_grid), r, c)
    
    # index of real captured img
    real_xy1 = find_idx_real()
    
    loss = loss_f(uv_cam,real_xy1)

    optimizer.zero_grad()
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    
    scheduler.step()
    
    if i % 1000 == 0:
            # print(depth)
            print(f" Depth value : {depth_value}, Epoch : {i}/{epoch}, Loss: {loss.item()}, LR: {optimizer.param_groups[0]['lr']}")


initialized depth value :  tensor([0.4082], device='cuda:0', requires_grad=True)
 Depth value : tensor([0.4182], device='cuda:0', requires_grad=True), Epoch : 0/10000, Loss: 131.22557067871094, LR: 0.01
 Depth value : tensor([1.4741], device='cuda:0', requires_grad=True), Epoch : 1000/10000, Loss: 54.57651901245117, LR: 0.005
 Depth value : tensor([1.4740], device='cuda:0', requires_grad=True), Epoch : 2000/10000, Loss: 54.57665252685547, LR: 0.0025


KeyboardInterrupt: 

## Diffraction grating calibration

#### camera pixel position from catured image

In [ ]:
# 특정 파장의 빛이 있는 camera pixel 위치 알아내기?

#### camera pixel position from unproj & proj method

#### Optimization